<a href="https://colab.research.google.com/github/AnamayaSaraogi/Eczema-detection/blob/main/eczema_detection_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Eczema Detection model created using MobileNet V2 because of less images in dataset.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import shutil
import random

# Set paths for the original dataset and new directories
base_dir = '/content/drive/MyDrive/eczema_dataset'
train_dir = '/content/drive/MyDrive/eczema_dataset/train'
val_dir = '/content/drive/MyDrive/eczema_dataset/val'

# Create directories for train and val splits
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(os.path.join(train_dir, 'eczema'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'normal'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'eczema'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'normal'), exist_ok=True)

# Function to split images into train/val
def split_data(source_dir, train_dir, val_dir, val_size=0.2):
    for category in ['eczema', 'normal']:
        category_dir = os.path.join(source_dir, category)
        files = os.listdir(category_dir)
        random.shuffle(files)
        val_count = int(len(files) * val_size)

        # Move files to train and val directories
        for file in files[val_count:]:
            shutil.move(os.path.join(category_dir, file), os.path.join(train_dir, category, file))
        for file in files[:val_count]:
            shutil.move(os.path.join(category_dir, file), os.path.join(val_dir, category, file))

# Split the data into train and val
split_data(base_dir, train_dir, val_dir)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the ImageDataGenerator for training with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,               # Normalize pixel values to [0, 1]
    rotation_range=40,            # Random rotations
    width_shift_range=0.2,        # Random horizontal shifts
    height_shift_range=0.2,       # Random vertical shifts
    shear_range=0.2,              # Random shear transformations
    zoom_range=0.2,               # Random zoom
    horizontal_flip=True,         # Random horizontal flips
    fill_mode='nearest'           # Strategy for filling newly created pixels
)

# Define the ImageDataGenerator for validation (no augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)

# Flow the images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),   # Resize all images to 150x150
    batch_size=32,
    class_mode='binary'       # Binary classification (eczema or normal)
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),   # Resize all images to 150x150
    batch_size=32,
    class_mode='binary'       # Binary classification (eczema or normal)
)


Found 22 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

# Load MobileNetV2 pre-trained on ImageNet, excluding the top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze all layers of MobileNetV2
base_model.trainable = False

# Build the model
model = models.Sequential([
    base_model,  # Add MobileNetV2 as the base
    layers.GlobalAveragePooling2D(),  # Global Average Pooling
    layers.Dense(1024, activation='relu'),  # Dense layer
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()


<ipython-input-5-1ba23cbfffb1>:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 5, 5, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       1,311,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │           1,025 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,570,753 (13.62 MB)

 Trainable params: 1,312,769 (5.01 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
# Train the model with frozen MobileNetV2 layers
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.5000 - loss: 1.3900 - val_accuracy: 0.5000 - val_loss: 5.3887
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.5000 - loss: 5.2242 - val_accuracy: 0.5000 - val_loss: 3.7100
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 598ms/step - accuracy: 0.5000 - loss: 3.5730 - val_accuracy: 0.5000 - val_loss: 0.6756
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - accuracy: 0.7273 - loss: 0.6428 - val_accuracy: 0.7500 - val_loss: 0.4019
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 596ms/step - accuracy: 0.8636 - loss: 0.3834 - val_accuracy: 0.5000 - val_loss: 1.4345
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 635ms/step - accuracy: 0.5455 - loss: 1.0658 - val_accuracy: 0.5000 - val_loss: 1.2493
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 606ms/step - accuracy: 0.6364 - loss: 0.9845 - val_accuracy: 0.7500 - val_loss: 0.4347
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 628ms/step - accuracy: 0.8182 - loss: 0.5517 - val_accuracy: 1.0000 - val_loss: 0.0

In [ ]:
# Unfreeze the top layers of MobileNetV2
base_model.trainable = True

# Freeze all layers except the last 4 layers of MobileNetV2
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Compile the model again with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_finetune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step - accuracy: 1.0000 - loss: 0.0351 - val_accuracy: 1.0000 - val_loss: 0.0286
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 746ms/step - accuracy: 0.9545 - loss: 0.0862 - val_accuracy: 1.0000 - val_loss: 0.0268
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step - accuracy: 1.0000 - loss: 0.0309 - val_accuracy: 1.0000 - val_loss: 0.0252
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step - accuracy: 1.0000 - loss: 0.0423 - val_accuracy: 1.0000 - val_loss: 0.0238
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 609ms/step - accuracy: 1.0000 - loss: 0.0302 - val_accuracy: 1.0000 - val_loss: 0.0225
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 634ms/step - accuracy: 0.9545 - loss: 0.0842 - val_accuracy: 1.0000 - val_loss: 0.0213
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step - accuracy: 1.0000 - loss: 0.0345 - val_accuracy: 1.0000 - val_loss: 0.0202
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 613ms/step - accuracy: 0.9545 - loss: 0.0549 - val_accuracy: 1.0000 - val_loss: 

In [ ]:
directory_path = "/content/drive/MyDrive/eczema_eyes/unseen_images"


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image

# Function to classify images recursively in a given directory
def classify_images_in_directory(directory_path):
    # Walk through all files and subdirectories in the directory
    for root, _, files in os.walk(directory_path):
        for file in files:
            # Check if the file is an image (you can add more image formats here if needed)
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                # Get the full image path
                img_path = os.path.join(root, file)

                # Load and preprocess the image
                img = image.load_img(img_path, target_size=(150, 150))  # Resize to match MobileNetV2 input size
                img_array = image.img_to_array(img)  # Convert to array
                img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

                # Predict the class
                prediction = model.predict(img_array)
                if prediction < 0.5:
                    print(f"{file}: Predicted class = Eczema")
                else:
                    print(f"{file}: Predicted class = Normal")

# Replace with the correct path to your image folder
directory_path = "/content/drive/MyDrive/eczema_eyes/unseen_images"  # Adjust path as needed
classify_images_in_directory(directory_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
eczema1.jpg: Predicted class = Eczema
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
eczema2.jpg: Predicted class = Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
eczema3.jpg: Predicted class = Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
eczema4.jpg: Predicted class = Eczema
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
eczema5.jpg: Predicted class = Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
noneczema1.jpg: Predicted class = Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
noneczema2.jpg: Predicted class = Normal


In [ ]:
model_save_path = "/content/drive/MyDrive/eczema_model.keras"  # Adjust path as needed
model.save(model_save_path)


Loading the model from google drive.

In [ ]:
from tensorflow.keras.models import load_model
model1 = load_model("/content/drive/MyDrive/eczema_model.keras")


Testing the model on unseen images.

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image

# Function to classify images recursively in a given directory
def classify_images_in_directory(directory_path):
    # Walk through all files and subdirectories in the directory
    for root, _, files in os.walk(directory_path):
        for file in files:
            # Check if the file is an image (you can add more image formats here if needed)
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                # Get the full image path
                img_path = os.path.join(root, file)

                # Load and preprocess the image
                img = image.load_img(img_path, target_size=(150, 150))  # Resize to match MobileNetV2 input size
                img_array = image.img_to_array(img)  # Convert to array
                img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

                # Predict the class
                prediction = model1.predict(img_array)
                if prediction < 0.5:
                    print(f"{file}: Predicted class = Eczema")
                else:
                    print(f"{file}: Predicted class = Normal")

# Replace with the correct path to your image folder
directory_path = "/content/drive/MyDrive/eczema_eyes/unseen_images"  # Adjust path as needed
classify_images_in_directory(directory_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
eczema6.jpg: Predicted class = Eczema
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
eczema7.jpg: Predicted class = Eczema
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
eczema8.jpg: Predicted class = Normal
